<a href="https://colab.research.google.com/github/minjae0501/yolo_block/blob/master/lstm_yolo_detecting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
# !git clone https://github.com/minjae0501/yolo_block.git

Cloning into 'yolo_block'...
remote: Enumerating objects: 3012, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 3012 (delta 3), reused 13 (delta 1), pack-reused 2993
Receiving objects: 100% (3012/3012), 117.88 MiB | 50.70 MiB/s, done.
Resolving deltas: 100% (50/50), done.


In [1]:
# %pip install ultralytics
# %pip install mediapipe
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.220 🚀 Python-3.9.0 torch-2.1.1 CPU (Apple M1 Pro)
Setup complete ✅ (10 CPUs, 16.0 GB RAM, 313.4/460.4 GB disk)


In [2]:
import cv2
import numpy as np
import mediapipe as mp
from ultralytics import YOLO
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import ImageFont, ImageDraw, Image

In [3]:
if torch.cuda.is_available() == True:
    device = 'cuda:0'
else:
    device = 'cpu'

print(device)

cpu


In [4]:
class MyDataset(Dataset):
    def __init__(self, seq_list):
        self.X = []
        self.y = []
        for dic in seq_list:
            self.y.append(dic['key'])
            self.X.append(dic['value'])
    def __getitem__(self, index):
        data = self.X[index]
        label = self.y[index]
        return torch.Tensor(np.array(data)), torch.tensor(np.array(int(label)))

    def __len__(self):
        return len(self.X)

In [5]:
class hand_LSTM(nn.Module):
    def __init__(self, num_layers=1):
        super(hand_LSTM, self).__init__()
        # LSTM 레이어 및 드롭아웃 레이어 정의
        self.lstm1 = nn.LSTM(67, 128, num_layers, batch_first=True) 
       
        self.lstm4 = nn.LSTM(128, 64, num_layers, batch_first=True)
        self.dropout4 = nn.Dropout(0.3)
        
        self.lstm5 = nn.LSTM(64, 32, num_layers, batch_first=True)
        self.dropout5 = nn.Dropout(0.3)
        self.fc = nn.Linear(32, 2)  # 최종 출력을 위한 선형 레이어

    def forward(self, x):
        # 순전파 정의
        x, _ = self.lstm1(x)
        x, _ = self.lstm4(x)
        x = self.dropout4(x)
        x, _ = self.lstm5(x)
        x = self.dropout5(x)
        x = self.fc(x[:, -1, :])  # 최종 시퀀스의 출력만 사용
        return x

In [6]:
def grab_release(image, yolo_model, hand_list , lstm_model, detect_cls, hand_cls, length, xyz_list_list, status, status_num):
    mp_hands, hands, mp_drawing = hand_list[0], hand_list[1], hand_list[2]

    # YOLO 객체 감지
    box_results = yolo_model.predict(image, conf = 0.6, verbose=False, show = False)
    boxes = box_results[0].boxes.xyxy.cpu()
    box_class = box_results[0].boxes.cls.cpu().tolist()

    x1, y1, x2, y2 = 0, 0, 0, 0
    hx1, hy1, hx2, hy2 = 0,0,0,0
    for idx, cls in enumerate(box_class):
        if int(cls) == detect_cls:
            x1, y1, x2, y2 = boxes[idx]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        elif int(cls) == hand_cls:
            hx1, hy1, hx2, hy2 = boxes[idx]
            hx1, hy1, hx2, hy2 = int(hx1), int(hy1), int(hx2), int(hy2)
            cv2.rectangle(image, (int(hx1), int(hy1)), (int(hx2), int(hy2)), (0, 0, 255), 2)
    
    #mediapipe
    results = hands.process(image)
    xyz_list = []
    if results.multi_hand_landmarks:
        for x_y_z in results.multi_hand_landmarks:
            for landmark in x_y_z.landmark:
                xyz_list.append(landmark.x * 10)
                xyz_list.append(landmark.y * 10)
                xyz_list.append(landmark.z * 10) # 
                

        xyz_list.append(abs(x1-hx1))
        xyz_list.append(abs(x2-hx2))
        xyz_list.append(abs(y1-hy1))
        xyz_list.append(abs(y2-hy2))

        if x1 != 0 and y1 != 0 and x2 != 0 and y2 != 0 and hx1 != 0 and hy1 != 0 and hx2 != 0 and hy2 != 0:
            xyz_list_list.append(xyz_list)# 객체와 손이 겹쳐야 

        for hand_landmarks in results.multi_hand_landmarks:
              with torch.no_grad():
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    
    if len(xyz_list_list) == length:
        dataset = []
        dataset.append({'key': 0, 'value': xyz_list_list})
        dataset = MyDataset(dataset)
        dataset = DataLoader(dataset)
        xyz_list_list = []
        for data, label in dataset:
            data = data.to(device)
            with torch.no_grad():
                result = lstm_model(data)
                _, out = torch.max(result, 1)
                status_num = out.item()
                if out.item() == 0: status = 'Release'
                else: status = 'Grab'

    # cv2.putText(image, status, (0, 50), cv2.FONT_HERSHEY_COMPLEX, 1.5, (0,0, 225), 2)

    return image, xyz_list_list, status, status_num



In [7]:
# 한글폰트 출력
def putText_korean(img, text, position, font_path, font_size, color):
    img_pil = Image.fromarray(img)
    draw = ImageDraw.Draw(img_pil)
    font = ImageFont.truetype(font_path, font_size)
    draw.text(position, text, font=font, fill=color)
    return np.array(img_pil)

In [8]:
# 실시간 안내 system 함수화
def process_step(image, step_model, lstm_model, yolo_model, hand_list, hand_cls, length, xyz_list_list, step_info):
    box_results = step_model.predict(image, conf=0.6, verbose=False, show=False)
    boxes = box_results[0].boxes.xyxy.cpu()
    box_class = box_results[0].boxes.cls.cpu().tolist()
    
    # 객체 감지 및 표시
    x1, y1, x2, y2 = 0, 0, 0, 0
    for idx, cls in enumerate(box_class):
        if int(cls) == step_info['step']:
            x1, y1, x2, y2 = map(int, boxes[idx])
            cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)
    
    text = step_info['default_text']
    if step_info['status'] == 'None':
        # 상태 업데이트
        image, xyz_list_list, status, status_num = grab_release(image, yolo_model, hand_list, lstm_model, step_info['detect_cls'], hand_cls, length, xyz_list_list, step_info['status'], step_info['status_num'])
        
        if status_num == 1:
            text = step_info['action_text']
            step_info['status'] = 'Detected'
        elif x1 != 0 and y1 != 0 and x2 != 0 and y2 != 0:
            step_info['cnt_frames'] += 1
            if step_info['cnt_frames'] >= step_info['wait_frames']:
                step_info['cnt_frames'] = 0
                step_info['status'] = 'Completed'

    elif step_info['status'] == 'Completed':
        text = '잘했어요!'
        return True, image, xyz_list_list, step_info, text  # 현재 단계 완료

    return False, image, xyz_list_list, step_info, text

In [9]:
# lstm모델 불러오기
model_path = './lstm_pth/more_data_lstm_model_ver2.pth'

# YOLO 객체 감지 모델 초기화
best_model = 'block_best_02.pt'
yolo_model = YOLO(best_model)

# YOLO 스탭 탐지 모델
step_best_model = 'truck_best_03.pt'
step_model = YOLO(step_best_model)

lstm_model = hand_LSTM().to(device)
lstm_model.load_state_dict(torch.load(model_path, map_location=device))
lstm_model.eval()
print("모델이 성공적으로 불러와졌습니다.")

모델이 성공적으로 불러와졌습니다.


In [10]:
# mediapipe 손 감지 모듈 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.3)
mp_drawing = mp.solutions.drawing_utils

hand_list = [mp_hands, hands, mp_drawing]

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [11]:
# 손, dataset길이 설정
hand_cls = 6
length = 5

# 한글 텍스트를 추가할 위치, 폰트경로, 폰트크기, 색상설정
position = (40, 40) # 텍스트를 출력할 위치
font_path = './font/KCC-Ganpan.ttf' # 한글 폰트 파일 경로
font_size = 30 # 폰트 크기
color = (0, 0, 255)

In [12]:
# 기본 설정 및 카메라 초기화
cap_device = 0
cap = cv2.VideoCapture(cap_device)
lstm_model.eval()
current_step = 0 # 현재 진행 중인 단계를 추적하는 변수
xyz_list_list = [] # 변수 초기화

steps = [
    {'step': 0, 'wait_frames': 60, 'cnt_frames': 0, 'detect_cls': 0, 'default_text': '파란 다리를 집어 올리세요.', 'action_text': '빨간 원 위에 파란 다리를 올려놓으세요.', 'status': 'None', 'status_num': -1},
    {'step': 1, 'wait_frames': 60, 'cnt_frames': 0, 'detect_cls': 13, 'default_text': '노란 다리를 집어 올리세요.', 'action_text': '파란색 다리의 왼쪽에 노란색 다리를 놓으세요.', 'status': 'None', 'status_num': -1},
    {'step': 2, 'wait_frames': 60, 'cnt_frames': 0, 'detect_cls': 4, 'default_text': '초록색 원을 집어 올리세요.', 'action_text': '노란색 다리 밑에 초록색 원을 넣어주세요.', 'status': 'None', 'status_num': -1},
    {'step': 3, 'wait_frames': 60, 'cnt_frames': 0, 'detect_cls': 5, 'default_text': '초록색 큐브를 집어 올리세요.', 'action_text': '노란색 다리의 오른쪽 위에 초록색 큐브를 올려놓으세요.', 'status': 'None', 'status_num': -1},
    {'step': 4, 'wait_frames': 60, 'cnt_frames': 0, 'detect_cls': 3, 'default_text': '파란색 부채꼴을 집어 올리세요.', 'action_text': '초록색 큐브의 왼쪽에 파란색 부채꼴을 놓으세요.', 'status': 'None', 'status_num': -1}
]

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    image = cv2.resize(frame, (640, 640))

    step_completed, image, xyz_list_list, step_info, text = process_step(
        image, step_model, lstm_model, yolo_model, hand_list, hand_cls, length, xyz_list_list, steps[current_step])

    if step_completed:
        if current_step < len(steps) - 1:
            current_step += 1  # 다음 단계로 이동
        else:
            break  # 모든 단계 완료

    image = putText_korean(image, text, position, font_path, font_size, color)
    cv2.imshow('frame', image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()
cv2.waitKey()

In [32]:
cap_device = 0
cap = cv2.VideoCapture(cap_device) # for Mac
# cap = cv2.VideoCapture(0) # for Windows
lstm_model.eval()
status = 'None'
status_num = -1
step = 0
wait_frames = 60
cnt_frames = 0
xyz_list_list = []
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    image = cv2.resize(frame, (640, 640))

    box_results = step_model.predict(image, conf = 0.6, verbose=False, show = False)
    boxes = box_results[0].boxes.xyxy.cpu()
    box_class = box_results[0].boxes.cls.cpu().tolist()

    x1, y1, x2, y2 = 0, 0, 0, 0
    for idx, cls in enumerate(box_class):
        if int(cls) == step:
            x1, y1, x2, y2 = boxes[idx]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (225, 0, 0), 2)
    
    if step == 0 :
        detect_cls = 0

        image, xyz_list_list, status, status_num = grab_release(image, yolo_model, hand_list , lstm_model, detect_cls, hand_cls, length, xyz_list_list, status, status_num)

        if status_num == 1:
            text = '빨간 원 위에 파란 다리를 올려놓으세요.'
        else:
            text = '파란 다리를 집어 올리세요.'
        

        if x1 != 0 and y1 != 0 and x2 != 0 and y2 != 0:
            if cnt_frames != wait_frames:
                text = '잘했어요!'
                cnt_frames += 1
            else:
                step += 1
                cnt_frames = 0
                status = 'None'
                status_num = -1
        image = putText_korean(image, text, position, font_path, font_size, color)

    elif step == 1:
        detect_cls = 13

        image, xyz_list_list, status, status_num = grab_release(image, yolo_model, hand_list , lstm_model, detect_cls, hand_cls, length, xyz_list_list, status, status_num)

        if status_num == 1:
            text = '파란색 다리의 왼쪽에 노란색 다리를 놓으세요.'
        else:
            text = '노란 다리를 집어 올리세요.'

        if x1 != 0 and y1 != 0 and x2 != 0 and y2 != 0:
            if cnt_frames != wait_frames:
                text = '잘했어요!'
                cnt_frames += 1
            else:
                step += 1
                cnt_frames = 0
                status = 'None'
                status_num = -1

        image = putText_korean(image, text, position, font_path, font_size, color)
    
    elif step == 2:
        detect_cls = 4

        image, xyz_list_list, status, status_num = grab_release(image, yolo_model, hand_list , lstm_model, detect_cls, hand_cls, length, xyz_list_list, status, status_num)

        if status_num == 1:
            text = '노란색 다리 밑에 초록색 원을 넣어주세요.'
        else:
            text = '초록색 원을 집어 올리세요.'

        if x1 != 0 and y1 != 0 and x2 != 0 and y2 != 0:
            if cnt_frames != wait_frames:
                text = '정말 잘 조립했어요!'
                cnt_frames += 1
            else:
                step += 1
                cnt_frames = 0
                status = 'None'
                status_num = -1
                
        image = putText_korean(image, text, position, font_path, font_size, color)
    
    elif step == 3:
        detect_cls = 5

        image, xyz_list_list, status, status_num = grab_release(image, yolo_model, hand_list , lstm_model, detect_cls, hand_cls, length, xyz_list_list, status, status_num)

        if status_num == 1:
            text = '노란색 다리의 오른쪽 위에 초록색 큐브를 올려놓으세요.'
        else:
            text = '초록색 큐브를 집어 올리세요.'

        if x1 != 0 and y1 != 0 and x2 != 0 and y2 != 0:
            if cnt_frames != wait_frames:
                text = '정말 잘 조립했어요!'
                cnt_frames += 1
            else:
                step += 1
                cnt_frames = 0
                status = 'None'
                status_num = -1
                
        image = putText_korean(image, text, position, font_path, font_size, color)
    
    elif step == 4:
        detect_cls = 3

        image, xyz_list_list, status, status_num = grab_release(image, yolo_model, hand_list , lstm_model, detect_cls, hand_cls, length, xyz_list_list, status, status_num)

        if status_num == 1:
            text = '초록색 큐브의 왼쪽에 파란색 부채꼴을 놓으세요.'
        else:
            text = '파란색 부채꼴을 집어 올리세요.'

        if x1 != 0 and y1 != 0 and x2 != 0 and y2 != 0:
            if cnt_frames != wait_frames:
                text = '완벽하게 해냈어요!'
                # cnt_frames += 1
            # else:
            #     step += 1
            #     cnt_frames = 0
            #     status = 'None'
            #     status_num = -1
                
        image = putText_korean(image, text, position, font_path, font_size, color)
    
    
    # image = cv2.resize(image, (1440, 640))
    cv2.imshow('frame',image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    
cap.release()
cv2.destroyAllWindows()
cv2.waitkey()

AttributeError: module 'cv2' has no attribute 'waitkey'

In [15]:
cap_device = 0
cap = cv2.VideoCapture(cap_device) # for Mac
# cap = cv2.VideoCapture(0) # for Windows
lstm_model.eval()
status = 'None'
status_num = -1
step = 0
wait_frames = 60
cnt_frames = 0
xyz_list_list = []
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    image = cv2.resize(frame, (640, 640))

    box_results = step_model.predict(image, conf = 0.6, verbose=False, show = False)
    boxes = box_results[0].boxes.xyxy.cpu()
    box_class = box_results[0].boxes.cls.cpu().tolist()

    x1, y1, x2, y2 = 0, 0, 0, 0
    for idx, cls in enumerate(box_class):
        if int(cls) == step:
            x1, y1, x2, y2 = boxes[idx]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (225, 0, 0), 2)
    
    if step == 0 :
        detect_cls = 0

        image, xyz_list_list, status, status_num = grab_release(image, yolo_model, hand_list , lstm_model, detect_cls, hand_cls, length, xyz_list_list, status, status_num)

        if status_num == 1:
            text = 'Put the blue bridge over the red circle'
        else:
            text = 'Pick up the blue bridge'
        

        if x1 != 0 and y1 != 0 and x2 != 0 and y2 != 0:
            if cnt_frames != wait_frames:
                text = 'Good Job!!'
                cnt_frames += 1
            else:
                step += 1
                cnt_frames = 0
                status = 'None'
                status_num = -1
        cv2.putText(image, text, (0, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0, 225), 2)

    elif step == 1:
        detect_cls = 13

        image, xyz_list_list, status, status_num = grab_release(image, yolo_model, hand_list , lstm_model, detect_cls, hand_cls, length, xyz_list_list, status, status_num)

        if status_num == 1:
            text = 'Put the yellow bridge on the left side of the blue bridge'
        else:
            text = 'Pick up the yellow bridge'

        if x1 != 0 and y1 != 0 and x2 != 0 and y2 != 0:
            if cnt_frames != wait_frames:
                text = 'Good Job!!'
                cnt_frames += 1
            else:
                step += 1
                cnt_frames = 0
                status = 'None'
                status_num = -1

        cv2.putText(image, text, (0, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0, 225), 2)
    
    elif step == 2:
        detect_cls = 4

        image, xyz_list_list, status, status_num = grab_release(image, yolo_model, hand_list , lstm_model, detect_cls, hand_cls, length, xyz_list_list, status, status_num)

        if status_num == 1:
            text = 'Put the green circle under the yellow bridge'
        else:
            text = 'Pick up the green circle'

        if x1 != 0 and y1 != 0 and x2 != 0 and y2 != 0:
            if cnt_frames != wait_frames:
                text = 'Good Job!!'
                cnt_frames += 1
            else:
                step += 1
                cnt_frames = 0
                status = 'None'
                status_num = -1
                
        cv2.putText(image, text, (0, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0, 225), 2)
    
    elif step == 3:
        detect_cls = 5

        image, xyz_list_list, status, status_num = grab_release(image, yolo_model, hand_list , lstm_model, detect_cls, hand_cls, length, xyz_list_list, status, status_num)

        if status_num == 1:
            text = 'Put the green cube on the top left of the yellow bridge'
        else:
            text = 'Pick up the green cube'

        if x1 != 0 and y1 != 0 and x2 != 0 and y2 != 0:
            if cnt_frames != wait_frames:
                text = 'Good Job!!'
                cnt_frames += 1
            else:
                step += 1
                cnt_frames = 0
                status = 'None'
                status_num = -1
                
        cv2.putText(image, text, (0, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0, 225), 2)
    
    elif step == 4:
        detect_cls = 3

        image, xyz_list_list, status, status_num = grab_release(image, yolo_model, hand_list , lstm_model, detect_cls, hand_cls, length, xyz_list_list, status, status_num)

        if status_num == 1:
            text = 'Put the blue quarter of circle on the right side of the green cube'
        else:
            text = 'Pick up the blue quarter of circle'

        if x1 != 0 and y1 != 0 and x2 != 0 and y2 != 0:
            if cnt_frames != wait_frames:
                text = 'Good Job!!'
                # cnt_frames += 1
            # else:
            #     step += 1
            #     cnt_frames = 0
            #     status = 'None'
            #     status_num = -1
                
        cv2.putText(image, text, (0, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0, 225), 2)
    
    
    # image = cv2.resize(image, (1440, 640))
    cv2.imshow('frame',image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    
cap.release()
cv2.destroyAllWindows()
cv2.waitkey()